In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Union, Callable
import random

In [6]:
import pandas
import pandas._version

print(pandas.__version__)

2.0.3


In [104]:
file_path = Path(r'C:\PythonProjects\social-media-preferences\03_Clustering_Marketing.csv')

In [105]:
df = pd.read_csv(file_path)

In [106]:
df.columns

Index(['gradyear', 'gender', 'age', 'NumberOffriends', 'basketball',
       'football', 'soccer', 'softball', 'volleyball', 'swimming',
       'cheerleading', 'baseball', 'tennis', 'sports', 'cute', 'sex', 'sexy',
       'hot', 'kissed', 'dance', 'band', 'marching', 'music', 'rock', 'god',
       'church', 'jesus', 'bible', 'hair', 'dress', 'blonde', 'mall',
       'shopping', 'clothes', 'hollister', 'abercrombie', 'die', 'death',
       'drunk', 'drugs'],
      dtype='object')

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 40 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   gradyear         15000 non-null  int64 
 1   gender           13663 non-null  object
 2   age              12504 non-null  object
 3   NumberOffriends  15000 non-null  int64 
 4   basketball       15000 non-null  int64 
 5   football         15000 non-null  int64 
 6   soccer           15000 non-null  int64 
 7   softball         15000 non-null  int64 
 8   volleyball       15000 non-null  int64 
 9   swimming         15000 non-null  int64 
 10  cheerleading     15000 non-null  int64 
 11  baseball         15000 non-null  int64 
 12  tennis           15000 non-null  int64 
 13  sports           15000 non-null  int64 
 14  cute             15000 non-null  int64 
 15  sex              15000 non-null  int64 
 16  sexy             15000 non-null  int64 
 17  hot              15000 non-null

In [108]:
df_int_stats = df.select_dtypes('int').describe()

In [109]:
for col in df.select_dtypes('object').columns:
    display(df[col].unique())

array([nan, 'F', 'M'], dtype=object)

array([nan, '17.41', '17.511', ..., '8.498', '19.975', '106.511'],
      dtype=object)

In [110]:
for col in df.select_dtypes('object').columns:
    display(f"Column '{col}', number of missing values: {df[col].isna().sum()}")


"Column 'gender', number of missing values: 1337"

"Column 'age', number of missing values: 2496"

In [111]:
df['age']\
    .where(df['age'].str.contains(pat='[a-z]'))\
        .sort_values(ascending=False)\
            .dropna(how='all')

14233     Mrz 63
2342     19. Nov
13826    19. Nov
13373    19. Nov
1157     19. Nov
          ...   
2721     15. Apr
6629     15. Apr
11351    15. Apr
5337     15. Apr
7550      14-Aug
Name: age, Length: 276, dtype: object

In [112]:
coin_flip(chanches=0.6)

False

In [113]:
df_int_stats

,gradyear,NumberOffriends,basketball,football,soccer,softball,volleyball,swimming,cheerleading,baseball,...,blonde,mall,shopping,clothes,hollister,abercrombie,die,death,drunk,drugs
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.00000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,...,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,2007.496933,29.834533,0.267000,0.255467,0.22200,0.159667,0.142933,0.135000,0.105133,0.104133,...,0.112867,0.259467,0.357267,0.150267,0.071800,0.051467,0.185867,0.118067,0.091733,0.061067
std,1.116516,35.386649,0.788851,0.702260,0.92042,0.737344,0.638747,0.548691,0.502491,0.519205,...,2.708619,0.704398,0.728512,0.478716,0.356258,0.280755,0.609928,0.455200,0.420631,0.349112
min,2006.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2006.000000,3.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2008.000000,20.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2008.000000,44.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2009.000000,605.000000,22.000000,9.000000,22.00000,17.000000,14.000000,31.000000,8.000000,14.000000,...,327.000000,12.000000,11.000000,8.000000,8.000000,8.000000,16.000000,14.000000,8.000000,16.000000


In [114]:
df_int_stats.loc[['min', 'mean', 'max', 'std'],]

,gradyear,NumberOffriends,basketball,football,soccer,softball,volleyball,swimming,cheerleading,baseball,...,blonde,mall,shopping,clothes,hollister,abercrombie,die,death,drunk,drugs
min,2006.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
mean,2007.496933,29.834533,0.267000,0.255467,0.22200,0.159667,0.142933,0.135000,0.105133,0.104133,...,0.112867,0.259467,0.357267,0.150267,0.071800,0.051467,0.185867,0.118067,0.091733,0.061067
max,2009.000000,605.000000,22.000000,9.000000,22.00000,17.000000,14.000000,31.000000,8.000000,14.000000,...,327.000000,12.000000,11.000000,8.000000,8.000000,8.000000,16.000000,14.000000,8.000000,16.000000
std,1.116516,35.386649,0.788851,0.702260,0.92042,0.737344,0.638747,0.548691,0.502491,0.519205,...,2.708619,0.704398,0.728512,0.478716,0.356258,0.280755,0.609928,0.455200,0.420631,0.349112


In [115]:
RAW_FEATURES = [
    "gradyear", "age", "NumberOffriends", "basketball",
    "football", "soccer", "softball", "volleyball", "swimming", "cheerleading",
    "baseball", "tennis", "sports", "cute", "sex", "sexy", "hot", "kissed",
    "dance", "band", "marching", "music", "rock", "god", "church", "jesus",
    "bible", "hair", "dress", "blonde", "mall", "shopping", "clothes",
    "hollister", "abercrombie", "die", "death", "drunk", "drugs", "gender"
]

In [116]:
def coin_flip(chanches: float = 0.80) -> bool:
    if random.random() < chanches:
        return True
    else:
        return False

In [142]:
def split_dataset(partitions: int, path: str = None, mock_issues: bool = False, shuffle: bool = False, chances_of_issues: float = 0.80) -> None:
    # Define folder/directory for data ingestion

    data_folder = Path('split_dataset\/')

    # Convert into Path the path argument
    source_file_path = Path(path)

    # Make sure to create the data_folder directories
    if not data_folder.exists(): data_folder.mkdir(parents=True, exist_ok=True)

    try:
        # Read the provided csv into a DataFrame
        df = pd.read_csv(source_file_path)

        if shuffle:
            df = df.sample(frac=1).reset_index()
            df = df.drop(columns=['index'])

        # Get the index of the last record
        max_index = int(df.tail(1).index[0])

        # Create equidistant data points from 0 to max index
        index_partitions = np.linspace(start=0, stop=max_index, num=partitions*2, dtype=np.int16).tolist()

        # Zip index partitions in pairs
        ordered_ranges = zip(index_partitions[0::2], index_partitions[1::2])

        # For every pair, create a csv file into the raw_data directory
        for pair in ordered_ranges:
            if mock_issues:
                generate_data_issues(dataframe=df.iloc[pair[0]:pair[1]], chances=chances_of_issues)\
                    .to_csv(path_or_buf=Path(data_folder / f'dataset_partition_{pair[0]}_{pair[1]}.csv'), 
                    index=False
                    )
            else:
                df.iloc[pair[0]:pair[1]].to_csv(
                    path_or_buf=Path(data_folder / f'dataset_partition_{pair[0]}_{pair[1]}.csv'), 
                    index=False
                )
    except Exception as e:
        print(f'Something wrong happened: {e}')

In [118]:
def produce_outliers(value: str | int | None) -> str | int:
    if str(value).isalpha():
        return value
    elif value is None:
        return value
    elif str(value).isnumeric() and value == 0:
        return np.random.randint(-100, -1)
    elif str(value).isnumeric():
        return -value

In [119]:
def set_string(value: str | int | None) -> str:
    if str(value).isalpha():
        return value
    elif str(value).isnumeric():
        return str(value)

In [120]:
def set_numeric(value: str | int | None) -> str | int:
    if str(value).isalpha():
        return value
    elif value is None:
        return pd.NA
    elif isinstance(value, str):
        return np.random.randint(0, 2009)

In [121]:
def new_categories(value: str) -> str:
    if value in ['F', 'M']:
        return np.random.choice(['B', 'NB', 'FTM', 'MTF', 'OTHER', ''])

In [122]:
def produce_nulls(value: str | int | None) -> pd.NA:
    value = pd.NA
    return value

In [123]:
issues_dict = {
    1: produce_nulls,
    2: new_categories,
    3: set_numeric,
    4: set_string,
    5: produce_outliers,
}

In [124]:
def apply_funcs(dataframe: pd.DataFrame, funcs: list[Callable], row: int) -> pd.DataFrame:
    random_affected_cols = np.random.choice(np.arange(0, 39), size=len(funcs), replace=False)
    for func, col  in zip(funcs, random_affected_cols):
        current_value = dataframe.iat[row, col]
        data_issue = func(current_value) 
        dataframe.iloc[[row], [col]] = data_issue
    return dataframe

In [125]:
def set_data_issues(dataframe: pd.DataFrame, affected_rows: dict) -> pd.DataFrame:
    check_key_value_match = [k for (k,v) in affected_rows.items() if k == v]

    if check_key_value_match:
        col_number = np.random.randint(0, 39)
        return dataframe.drop(df.columns[col_number], axis=1)

    else:
        for k, v in affected_rows.items():
            pick_random_issue = np.random.choice(np.arange(1, 6), size=5, replace=False)
            funcs = [func for (k, func) in issues_dict.items() if k in pick_random_issue]
            dataframe = apply_funcs(dataframe, funcs, row=k)
            
        return dataframe

In [126]:
def determine_affected_rows(dataframe: pd.DataFrame) -> dict:
    number_of_rows = dataframe.shape[0] - 1
    afected_rows = np.random.randint(1, number_of_rows, size=number_of_rows)
    set_issues_dict = {}

    for idx in afected_rows:
        if idx not in set_issues_dict.keys():
            set_issues_dict[idx] = 1
        else:
            set_issues_dict[idx] += 1
    return {k:v for (k, v) in set_issues_dict.items() if v > 1}

In [127]:
def generate_data_issues(dataframe: pd.DataFrame = None, chances: float = None) -> pd.DataFrame:
    if coin_flip(chances):
        affected_rows = determine_affected_rows(dataframe)
        dataframe = set_data_issues(dataframe, affected_rows)
        return dataframe
    else:
        return dataframe

In [128]:
df_test = df.sample(25)

In [129]:
df_test_ouput = generate_data_issues(dataframe=df_test, chances=0.01)

In [144]:
split_dataset(partitions=100, path='03_Clustering_Marketing.csv', mock_issues=True, shuffle=True, chances_of_issues=0.1)